# Decision Tree Plots


## TODOs

- adapt plots, columns plotted not applicable to my experiment!
- TODO do column plots with subplots (to save space, no y labels etc!)
- TODO show line style in legends
- TODO add legend background

In [2]:
# global settings
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.interpolate import griddata
import numpy as np
import seaborn as sns
import pandas as pd
import itertools
import matplotlib.ticker as ticker
sns.set_style("ticks")
sns.set_context({"figure.figsize": (4.5, 3)})

## Load data

(Data files can be multiple GB. If files are small then use uncompressed files --> faster!)

In [3]:
#print(np.version.version) # numpy version 1.15.2
print("<--- Test experiment --->")
%time df = pd.read_pickle("data/test.pkl", compression="bz2")  # -r 100
print(df)

print("\n<--- DT experiment --->")
%time df1 = pd.read_pickle("data/DT_selection.pkl", compression="bz2")  # -r 100
print(df1)

<--- Test experiment --->
CPU times: user 11 ms, sys: 5.86 ms, total: 16.8 ms
Wall time: 18.3 ms
     alpha  conf_id  degree  epsilon   error  error_value  k_samples  \
0      0.1        1       2      0.1     MSE     2.722046          2   
1      0.1        1       2      0.1    RMSD     1.649862          2   
2      0.1        1       2      0.1  NRMSDM     0.776862          2   
3      0.1        1       2      0.1  NRMSDR     0.333306          2   
4      0.1        1       2      0.1     MSE     2.342276          2   
5      0.1        1       2      0.1    RMSD     1.530450          2   
6      0.1        1       2      0.1  NRMSDM     0.720635          2   
7      0.1        1       2      0.1  NRMSDR     0.309182          2   
8      0.1        1       2      0.1     MSE     2.296527          2   
9      0.1        1       2      0.1    RMSD     1.515430          2   
10     0.1        1       2      0.1  NRMSDM     0.713562          2   
11     0.1        1       2      0.1  N